In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 12


# import seaborn as sns

# from imblearn.over_sampling import SMOTE

In [35]:
df = pd.read_csv("/Users/ianchang/0.Workspaces/slaCoefficientProject/Origination Data/DTT SLA Summary - Overall Summary.csv").drop(columns=["Unnamed: 0", "partner_id", "partner", "xfr_request_id", "request_id", "true_SLA", "runtime_per_day", "payment_code"])

In [36]:
df['submit_date'] = pd.to_datetime(df['submit_date'])
df["submit_date"] = df["submit_date"].dt.floor('D')

# Extract year and month, then create the submit_month column
df['submit_month'] = df['submit_date'].dt.to_period('M')

In [37]:
origination_by_month = df[["on_time", "submit_month"]].groupby(by="submit_month").mean().reset_index()


In [38]:
# fig, ax = plt.subplots(figsize = (10,5))
# plt.style.use('bmh')

# ax.plot(origination_by_month["submit_month"].astype("str"), origination_by_month["on_time"], marker="v", label="Monthly OTD%", color="blue")


# ax.set_ylim(0.9, 1)
# ax.axhline(y=0.95, color='red', linestyle='--', label="Desired Level of OTD%")
# ax.set_title("Monthly Trend of On Time Delivery Percentage (OTD%)")
# ax.set_xlabel("Month")
# ax.set_ylabel("% in Decimals")

# plt.xticks(rotation=45)
# plt.legend()
# plt.show()

In [39]:
import random
import string

def get_random_string(length=8):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

df['author'] = df['author'].str.replace(r'@.*', '', regex=True)

unique_authors = df['author'].unique()

author_mapping = {}

for author in unique_authors:
    author_mapping[author] = get_random_string()

df['author_anonymous'] = df['author'].map(author_mapping)



In [40]:
# df = df.sort_values(by="author_anonymous")

In [41]:
# df.groupby(by=["author_anonymous", "asset_type", "reason_code", "source_language", "target_language"]).agg({"runtime_minutes": "sum"}).head(50)

## Service Level Agreement Based on Asset Types:

● Standard SLA for all subtitles and language templates: 15 RTM per Day

● Rush SLA for all subtitles and language templates: 30 RTM Day

● Standard SLA for CC for Original Audio language: 30 RTM per Day

● Rush SLA for CC for Original Audio language: 60 RTM per Day

● Standard SLA for CC for Non Original Audio language: 15 RTM per Day

● Rush SLA for CC for Non Original Audio language: 30 RTM per Day

In [42]:
# Initialize  
df["expected_daily_output"] = 0

for index, row in df.iterrows():
    if row["asset_type"] == "SUBTITLE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

    elif row["asset_type"] == "CLOSED_CAPTIONING":
        if row["original_audio_language"] == row["target_language"]:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 30
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 60
        else:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 15
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 30
    
    elif row["asset_type"] == "LANGUAGE_TEMPLATE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

df = df[['author_anonymous', 'package_id', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date', 'due_date', 'on_time',]]

In [45]:
import pandas as pd

def calculate_business_days(start_date_str, end_date_str):
    """
    Calculate the number of business days between two dates excluding weekends.

    Parameters:
    start_date_str (str): The start date in YYYY-MM-DD format.
    end_date_str (str): The end date in YYYY-MM-DD format.

    Returns:
    int: The number of business days between the two dates.
    """
    
    # Convert the date strings to datetime objects
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    
    # Calculate the number of business days (excluding weekends)
    business_days = pd.date_range(start=start_date, end=end_date, freq='B')
    
    # Return the count of business days
    return len(business_days)


In [46]:
import math

for index, row in df.iterrows():
    df.at[index,"num_days_spent"] = calculate_business_days(row["start_date"], row["submit_date"])
    df.at[index, "expected_TAT"] = np.ceil(row["runtime_minutes"] / row["expected_daily_output"])
    df.at[index, "sla_coef"] = row["num_days_spent"] / row["expected_TAT"]

df = df[['author_anonymous', 'package_id', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date', 'due_date', 'on_time', 'num_days_spent','expected_TAT', "sla_coef"]]

In [48]:
df

,author_anonymous,package_id,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,expected_daily_output,runtime_minutes,start_date,submit_date,due_date,on_time,num_days_spent,expected_TAT,sla_coef
0,WbEDyYdS,2096846,LANGUAGE_TEMPLATE,VIDEO,RUSH,en,--,en,30,77.6,2022-12-30,2023-01-03,2023-01-05,1,3.0,3.0,1.0
1,zZaOIMzj,2096828,SUBTITLE,VIDEO,RUSH,en,en,ko,30,50.6,2023-01-01,2023-01-03,2023-01-04,1,2.0,2.0,NaN
2,yZDHLuhx,2081320,CLOSED_CAPTIONING,VIDEO,RUSH,en,en,cs,30,56.2,2022-12-29,2023-01-03,2023-01-07,1,4.0,2.0,NaN
3,nBvhcfUc,2097639,LANGUAGE_TEMPLATE,VIDEO,RUSH,en,--,en,30,46.9,2022-12-31,2023-01-03,2023-01-03,1,2.0,2.0,NaN
4,JHdQkRvC,2081336,CLOSED_CAPTIONING,VIDEO,RUSH,en,en,cs,30,55.0,2022-12-31,2023-01-04,2023-01-07,1,3.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12758,iXWVFbOH,2119758,SUBTITLE,VIDEO,STANDARD,en,en,nb,15,92.1,2022-12-22,2023-01-01,2023-01-02,1,7.0,7.0,NaN
12759,wLduXFrK,2097180,SUBTITLE,VIDEO,STANDARD,it,en,zh-Hant,15,42.8,2022-12-27,2023-01-01,2023-01-03,1,4.0,3.0,NaN
12760,jIKrHtZE,2132627,CLOSED_CAPTIONING,VIDEO,STANDARD,en,en,en,30,29.7,2022-12-30,2023-01-01,2023-01-02,1,1.0,1.0,NaN
12761,aWatvAHz,2096062,SUBTITLE,VIDEO,STANDARD,en,en,zh-Hans,15,24.1,2022-12-30,2023-01-01,2023-01-17,1,1.0,2.0,NaN


In [32]:
# fig, ax = plt.subplots(figsize= (10,5))

# ax.hist(df["sla_coef"], bins=100)

# plt.show()

In [49]:
df.to_csv('df_newTAT.csv', index=False)